In [1]:
import torch
import torch.nn.functional as F
from torch.optim.lr_scheduler import StepLR, MultiStepLR
import numpy as np
import torch.nn as nn
from math import *

In [2]:
torch.cuda.set_device(1)

In [3]:
torch.set_default_tensor_type('torch.DoubleTensor') # 设置浮点类型为 torch.float64

In [4]:
# 定义激活函数: swish(x)
def acti(x):
    return x*torch.sigmoid(x) 

In [5]:
# 定义网络结构
class DeepRitzNet(torch.nn.Module):
    def __init__(self, input_width, layer_width):
        super(DeepRitzNet, self).__init__()
        self.linear_in = torch.nn.Linear(input_width, layer_width)
        self.linear1 = torch.nn.Linear(layer_width, layer_width)
        self.linear2 = torch.nn.Linear(layer_width, layer_width)
        self.linear3 = torch.nn.Linear(layer_width, layer_width)
        self.linear4 = torch.nn.Linear(layer_width, layer_width)
        self.linear5 = torch.nn.Linear(layer_width, layer_width)
        self.linear6 = torch.nn.Linear(layer_width, layer_width)
        self.linear_out = torch.nn.Linear(layer_width, 1)

    def forward(self, x):
        y = self.linear_in(x) # fully connect layer
        y = y + acti(self.linear2(acti(self.linear1(y)))) # residual block 1
        y = y + acti(self.linear4(acti(self.linear3(y)))) # residual block 2
        y = y + acti(self.linear6(acti(self.linear5(y)))) # residual block 3
        output = self.linear_out(y) # fully connect layer
        return output

In [6]:
dimension = 4

In [7]:
# exact solution
def u_ex(x):  
    x_temp = torch.cos(pi*x)
    u_x = (x_temp.sum(1)).reshape([x.size()[0], 1]) # x_temp.sum(1) 按行求和
    return u_x

In [8]:
def f(x):
    x_temp = torch.cos(pi*x)
    f_x = 2*pi**2*(x_temp.sum(1)).reshape([x.size()[0], 1]) # x_temp.sum(1) 按行求和
    return f_x

In [9]:
def DRM(x):
    u_hat = model(x)
    ux = torch.zeros(x.size()[0], dimension).cuda()
    step_size = 0.0001
    for i in range(dimension):
        dx = torch.zeros(x.size()[0], dimension).cuda()
        dx[:, i] = torch.ones(x.size()[0])
        ux[:, i] = (model(x+step_size*dx) - model(x-step_size*dx))[:,0]/step_size/2
    uxsq = (torch.sum(ux**2, dim = 1)).reshape([x.size()[0], 1]) # dim = 1 按行求和
    f_temp = f(x)
    part_1 = torch.sum(0.5 * uxsq + 0.5*pi**2*u_hat**2 - f_temp*u_hat)/x.size()[0]
    
    Nb = 100
    xa1 = torch.rand(Nb, dimension).cuda() 
    xa1[:, 0] = torch.zeros(Nb)
    xa2 = torch.rand(Nb, dimension).cuda() 
    xa2[:, 0] = torch.ones(Nb)
    xb1 = torch.rand(Nb, dimension).cuda()
    xb1[:, 1] = torch.zeros(Nb)
    xb2 = torch.rand(Nb, dimension).cuda()
    xb2[:, 1] = torch.ones(Nb)
    xc1 = torch.rand(Nb, dimension).cuda()
    xc1[:, 2] = torch.zeros(Nb)
    xc2 = torch.rand(Nb, dimension).cuda()
    xc2[:, 2] = torch.ones(Nb)
    xd1 = torch.rand(Nb, dimension).cuda()
    xd1[:, 3] = torch.zeros(Nb)
    xd2 = torch.rand(Nb, dimension).cuda()
    xd2[:, 3] = torch.ones(Nb)
    sa1 = (torch.sum((model(xa1)-u_ex(xa1))**2)/xa1.size()[0]).cuda()
    sa2 = (torch.sum((model(xa2)-u_ex(xa2))**2)/xa1.size()[0]).cuda() 
    sb1 = (torch.sum((model(xb1)-u_ex(xb1))**2)/xa1.size()[0]).cuda() 
    sb2 = (torch.sum((model(xb2)-u_ex(xb2))**2)/xa1.size()[0]).cuda()
    sc1 = (torch.sum((model(xc1)-u_ex(xc1))**2)/xa1.size()[0]).cuda() 
    sc2 = (torch.sum((model(xc2)-u_ex(xc2))**2)/xa1.size()[0]).cuda()
    sd1 = (torch.sum((model(xd1)-u_ex(xd1))**2)/xa1.size()[0]).cuda() 
    sd2 = (torch.sum((model(xd2)-u_ex(xd2))**2)/xa1.size()[0]).cuda()
    part_2 = sa1 + sa2 + sb1 + sb2 + sc1 + sc2 + sd1 + sd2
    
    lambda1 = 100.0
    return part_1 + lambda1 * part_2 / 8

In [10]:
Data_size = 2000
def Gendata():
    x = torch.rand(Data_size, dimension)
    return x.cuda()

In [11]:
# 正态分布初始化参数
def initparam(model, sigma):
    for m in model.modules():
        if isinstance(m, nn.Linear):
            m.weight.data.normal_(0, sigma)
    return model

In [12]:
model = DeepRitzNet(dimension, 8)
model = initparam(model, 0.5)

In [13]:
device = torch.device("cuda:1" )
model.to(device)

DeepRitzNet(
  (linear_in): Linear(in_features=4, out_features=8, bias=True)
  (linear1): Linear(in_features=8, out_features=8, bias=True)
  (linear2): Linear(in_features=8, out_features=8, bias=True)
  (linear3): Linear(in_features=8, out_features=8, bias=True)
  (linear4): Linear(in_features=8, out_features=8, bias=True)
  (linear5): Linear(in_features=8, out_features=8, bias=True)
  (linear6): Linear(in_features=8, out_features=8, bias=True)
  (linear_out): Linear(in_features=8, out_features=1, bias=True)
)

In [14]:
import torch.optim as optim
import torch.nn as nn
import time

In [15]:
def relative_error():
    x = Gendata()
    predict = model(x)
    exact = u_ex(x)
    value = torch.sqrt(torch.sum((predict - exact )**2))/torch.sqrt(torch.sum((exact )**2))
    return value

In [16]:
traintime = 20000
error_save = np.zeros(traintime)
optimizer = optim.Adam(model.parameters())

In [17]:
time_start = time.time()
for i in range(traintime):
    optimizer.zero_grad()
    x = Gendata()
    x.requires_grad = True
    losses = DRM(x)
    losses.backward()
    optimizer.step()
    error = relative_error()
    error_save[i] = float(error)
    
    if i % 50 == 0:
        print("current epoch is: ", i)
        print("current loss is: ", losses.detach())
        print("current relative error is: ", error.detach())
        np.save("DRM_relative_error_4D_Dirichlet_lambda100.0.npy", error_save)
np.save("DRM_relative_error_4D_Dirichlet_lambda100.0.npy", error_save)
time_end = time.time()
print('total time is: ', time_end-time_start, 'seconds')

current epoch is:  0
current loss is:  tensor(1313.1007, device='cuda:1')
current relative error is:  tensor(2.0970, device='cuda:1')
current epoch is:  50
current loss is:  tensor(371.9866, device='cuda:1')
current relative error is:  tensor(1.2116, device='cuda:1')
current epoch is:  100
current loss is:  tensor(298.8694, device='cuda:1')
current relative error is:  tensor(1.0759, device='cuda:1')
current epoch is:  150
current loss is:  tensor(226.1879, device='cuda:1')
current relative error is:  tensor(0.9539, device='cuda:1')
current epoch is:  200
current loss is:  tensor(105.7057, device='cuda:1')
current relative error is:  tensor(0.7149, device='cuda:1')
current epoch is:  250
current loss is:  tensor(11.1438, device='cuda:1')
current relative error is:  tensor(0.3665, device='cuda:1')
current epoch is:  300
current loss is:  tensor(-3.0229, device='cuda:1')
current relative error is:  tensor(0.2560, device='cuda:1')
current epoch is:  350
current loss is:  tensor(-6.6562, de

current epoch is:  3050
current loss is:  tensor(-17.3165, device='cuda:1')
current relative error is:  tensor(0.0841, device='cuda:1')
current epoch is:  3100
current loss is:  tensor(-17.3831, device='cuda:1')
current relative error is:  tensor(0.0826, device='cuda:1')
current epoch is:  3150
current loss is:  tensor(-17.1136, device='cuda:1')
current relative error is:  tensor(0.0831, device='cuda:1')
current epoch is:  3200
current loss is:  tensor(-18.2926, device='cuda:1')
current relative error is:  tensor(0.0811, device='cuda:1')
current epoch is:  3250
current loss is:  tensor(-19.1044, device='cuda:1')
current relative error is:  tensor(0.0808, device='cuda:1')
current epoch is:  3300
current loss is:  tensor(-17.2221, device='cuda:1')
current relative error is:  tensor(0.0837, device='cuda:1')
current epoch is:  3350
current loss is:  tensor(-19.1569, device='cuda:1')
current relative error is:  tensor(0.0814, device='cuda:1')
current epoch is:  3400
current loss is:  tensor

current epoch is:  6100
current loss is:  tensor(-19.0098, device='cuda:1')
current relative error is:  tensor(0.0227, device='cuda:1')
current epoch is:  6150
current loss is:  tensor(-19.0514, device='cuda:1')
current relative error is:  tensor(0.0216, device='cuda:1')
current epoch is:  6200
current loss is:  tensor(-20.7643, device='cuda:1')
current relative error is:  tensor(0.0208, device='cuda:1')
current epoch is:  6250
current loss is:  tensor(-20.5306, device='cuda:1')
current relative error is:  tensor(0.0219, device='cuda:1')
current epoch is:  6300
current loss is:  tensor(-19.5653, device='cuda:1')
current relative error is:  tensor(0.0212, device='cuda:1')
current epoch is:  6350
current loss is:  tensor(-19.1591, device='cuda:1')
current relative error is:  tensor(0.0212, device='cuda:1')
current epoch is:  6400
current loss is:  tensor(-19.3228, device='cuda:1')
current relative error is:  tensor(0.0206, device='cuda:1')
current epoch is:  6450
current loss is:  tensor

current epoch is:  9150
current loss is:  tensor(-20.0796, device='cuda:1')
current relative error is:  tensor(0.0135, device='cuda:1')
current epoch is:  9200
current loss is:  tensor(-21.4624, device='cuda:1')
current relative error is:  tensor(0.0132, device='cuda:1')
current epoch is:  9250
current loss is:  tensor(-18.5286, device='cuda:1')
current relative error is:  tensor(0.0172, device='cuda:1')
current epoch is:  9300
current loss is:  tensor(-19.2563, device='cuda:1')
current relative error is:  tensor(0.0166, device='cuda:1')
current epoch is:  9350
current loss is:  tensor(-19.9435, device='cuda:1')
current relative error is:  tensor(0.0142, device='cuda:1')
current epoch is:  9400
current loss is:  tensor(-19.8905, device='cuda:1')
current relative error is:  tensor(0.0198, device='cuda:1')
current epoch is:  9450
current loss is:  tensor(-21.3873, device='cuda:1')
current relative error is:  tensor(0.0161, device='cuda:1')
current epoch is:  9500
current loss is:  tensor

current epoch is:  12150
current loss is:  tensor(-21.0373, device='cuda:1')
current relative error is:  tensor(0.0097, device='cuda:1')
current epoch is:  12200
current loss is:  tensor(-20.7600, device='cuda:1')
current relative error is:  tensor(0.0112, device='cuda:1')
current epoch is:  12250
current loss is:  tensor(-19.3608, device='cuda:1')
current relative error is:  tensor(0.0094, device='cuda:1')
current epoch is:  12300
current loss is:  tensor(-20.3486, device='cuda:1')
current relative error is:  tensor(0.0108, device='cuda:1')
current epoch is:  12350
current loss is:  tensor(-19.6966, device='cuda:1')
current relative error is:  tensor(0.0132, device='cuda:1')
current epoch is:  12400
current loss is:  tensor(-19.2392, device='cuda:1')
current relative error is:  tensor(0.0095, device='cuda:1')
current epoch is:  12450
current loss is:  tensor(-19.1866, device='cuda:1')
current relative error is:  tensor(0.0102, device='cuda:1')
current epoch is:  12500
current loss is:

current epoch is:  15150
current loss is:  tensor(-20.8378, device='cuda:1')
current relative error is:  tensor(0.0141, device='cuda:1')
current epoch is:  15200
current loss is:  tensor(-19.5099, device='cuda:1')
current relative error is:  tensor(0.0087, device='cuda:1')
current epoch is:  15250
current loss is:  tensor(-20.8432, device='cuda:1')
current relative error is:  tensor(0.0087, device='cuda:1')
current epoch is:  15300
current loss is:  tensor(-19.7479, device='cuda:1')
current relative error is:  tensor(0.0179, device='cuda:1')
current epoch is:  15350
current loss is:  tensor(-19.0564, device='cuda:1')
current relative error is:  tensor(0.0091, device='cuda:1')
current epoch is:  15400
current loss is:  tensor(-20.5623, device='cuda:1')
current relative error is:  tensor(0.0080, device='cuda:1')
current epoch is:  15450
current loss is:  tensor(-17.7401, device='cuda:1')
current relative error is:  tensor(0.0080, device='cuda:1')
current epoch is:  15500
current loss is:

current epoch is:  18150
current loss is:  tensor(-19.9857, device='cuda:1')
current relative error is:  tensor(0.0073, device='cuda:1')
current epoch is:  18200
current loss is:  tensor(-19.7071, device='cuda:1')
current relative error is:  tensor(0.0098, device='cuda:1')
current epoch is:  18250
current loss is:  tensor(-21.1448, device='cuda:1')
current relative error is:  tensor(0.0084, device='cuda:1')
current epoch is:  18300
current loss is:  tensor(-19.2349, device='cuda:1')
current relative error is:  tensor(0.0076, device='cuda:1')
current epoch is:  18350
current loss is:  tensor(-17.4558, device='cuda:1')
current relative error is:  tensor(0.0085, device='cuda:1')
current epoch is:  18400
current loss is:  tensor(-20.8890, device='cuda:1')
current relative error is:  tensor(0.0079, device='cuda:1')
current epoch is:  18450
current loss is:  tensor(-20.3264, device='cuda:1')
current relative error is:  tensor(0.0083, device='cuda:1')
current epoch is:  18500
current loss is: